In [20]:
from google.colab import drive
drive.mount('/content/drive')
fname = "/content/drive/My Drive/Colab Notebooks/iris.csv.gz"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [22]:
dataset = pd.read_csv(fname)

In [23]:
# Срез: отсекаем классы, учитываем вес w0: добавляем столбец единиц
x = dataset.values[:, 0:4]
x = np.asarray([np.append(i, 1.0) for i in x])

In [39]:
# Оставляем только классы
y = dataset.values[:, 4]
y_norm = np.asarray([-1 if i == 'virginica' else 1 for i in y])

In [41]:
# Перемешать и разделить данные в пропорции 90:10
x_train, x_test, y_train, y_test = train_test_split(x,y_norm, test_size=0.2)

In [28]:
def soft_margin_loss(image_x, image_y):
  return max(0.1 - image_y * np.dot(x, weights)) + alpha * np.dot(weights, weights)

In [29]:
def predict(x_test, weights):
  y_prediction = []
  for i in x_test:
    y_prediction.append(np.sign(np.dot(weights, i)))
  return np.asarray(y_prediction)

In [30]:
# 
def margin(i, y_train, weights, x_train):
  return y_train[i] * np.dot(weights, x_train[i])

In [31]:
# 
def update_weights(x_train, y_train, margin, i, weights):
  return weights - learn_rate * alpha * weights if margin >=1 else weights + (learn_rate*(y_train[i] * x_train[i] - alpha * weights))

In [43]:
# SVM

epochs = 50
learn_rate = 0.01
# быстрота прропорционального уменьшеная весов (регуляризация)
alpha = 0.1

# Возможно лучше нормальное распределение
weights = np.asarray([random.random() for i in range(x_train.shape[1])])
print("Начальное распределение весов: ", weights)

for epoch in range(epochs):
  loss = 0.0
  for i, x_image in enumerate(x_train):
    margin_ = margin(i, y_train, weights, x_train)
    weights = update_weights(x_train, y_train, margin_, i, weights)
    loss += soft_margin_loss(x_train[i], y_train[i])

print("Обновленное распределение весов: ", weights)

print("Результат предсказания: ", predict(x_test, weights))
print("Реальное значение: ", y_test)
loss = 0.0
for pred, real in zip(predict(x_test, weights), y_test):
  if pred != real:
    loss += 1
loss /= (len(y_test) / 100)
print("Ошибка: ", loss, "%")

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(x_train, y_train)

print("При помощи библиотеки: ", clf.predict(x_test))
loss = 0.0
for pred, real in zip(clf.predict(x_test), y_test):
  if pred != real:
    loss += 1
loss /= (len(y_test) / 100)
print("Ошибка: ", loss, "%")

Начальное распределение весов:  [0.54116888 0.99286342 0.1805107  0.58847746 0.55302077]
Обновленное распределение весов:  [0.5933117387736365 0.5186674528836115 -0.8353097075242902
 -0.7734712479789244 0.2754103185582833]
Результат предсказания:  [ 1. -1.  1. -1. -1. -1. -1.  1.  1.  1. -1. -1.  1. -1. -1.  1.  1.  1.
  1.  1. -1.  1. -1.  1. -1.  1. -1.  1.  1.  1.]
Реальное значение:  [ 1 -1  1 -1 -1 -1 -1  1  1  1 -1 -1  1 -1 -1  1  1  1  1  1 -1  1 -1  1
 -1  1 -1 -1  1  1]
Ошибка:  3.3333333333333335 %
При помощи библиотеки:  [ 1 -1  1 -1 -1 -1 -1  1  1  1 -1 -1  1 -1 -1  1  1  1  1  1 -1  1 -1  1
 -1  1 -1 -1  1  1]
Ошибка:  0.0 %
